In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 1.3 MB/s eta 0:01:37
   ---------------------------------------- 0.8/124.9 MB 1.3 MB/s eta 0:01:33
   ---------------------------------------- 1.3/124.9 MB 1.5 MB/s eta 0:01:25
    --------------------------------------- 1.6/124.9 MB 1.4 MB/s eta 0:01:30
    --------------------------------------- 1.8/124.9 MB 1.4 MB/s eta 0:01:30
    --------------------------------------- 2.1/124.9 MB 1.4 MB/s eta 0:01:28
    --------------------------------------- 2.4/124.9 MB 1.4 MB/s eta 0:01:29
    --------------------------------------- 2.9/124.9 MB 1.4 MB/s eta 0:01:25
   - -------------------------------------- 3.4/124.9 MB 1.5 MB/s eta 0:01:19
   - -------------------------------------- 4.2/124.9 MB 1.7 MB/s eta 0:01:11
   - --

In [6]:
train_data = pd.read_csv('./dataset/twitter_training.csv', header=None)
val_data = pd.read_csv('./dataset/twitter_validation.csv', header=None)

In [7]:
train_data.columns = ['id', 'information', 'type', 'text']
train_data

,id,information,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [8]:
val_data.columns = ['id', 'information', 'type', 'text']
val_data

,id,information,type,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [9]:
train_data['lower'] = train_data['text'].str.lower()
train_data['lower'] = [str(data) for data in train_data['lower']]
train_data['lower'] = train_data['lower'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

train_data

,id,information,type,text,lower
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...
...,...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,just realized that the windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,just realized that my mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,just realized the windows partition of my mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,just realized between the windows partition of...


In [10]:
val_data['lower'] = val_data['text'].str.lower()
val_data['lower'] = [str(data) for data in val_data['lower']]
val_data['lower'] = val_data['lower'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

val_data

,id,information,type,text,lower
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...,i mentioned on facebook that i was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...,bbc news amazon boss jeff bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...,microsoft why do i pay for word when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,...",csgo matchmaking is so full of closet hacking ...
4,4433,Google,Neutral,Now the President is slapping Americans in the...,now the president is slapping americans in the...
...,...,...,...,...,...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...,toronto is the arts and culture capital of c...
996,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...,this is actually a good move tot bring more vi...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...,today sucked so it s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.,bought a fraction of microsoft today small wins


In [ ]:
tokens_text = [word_tokenize(str(word)) for word in train_data['lower']]
tokens_counter = [item for sublist in tokens_text for item in sublist]

print(len(set(tokens_counter)))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\nltk_data'
    - 'C:\\Users\\ASUS/nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [13]:
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')

In [14]:
bow_count = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=stop_words,
    ngram_range=(1, 1)
)

In [16]:
reviews_train, reviews_test = train_test_split(train_data, test_size=0.2, random_state=0)

In [17]:
X_train_bow = bow_count.fit_transform(reviews_train['lower'])
X_test_bow = bow_count.transform(reviews_test['lower'])

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\nltk_data'
    - 'C:\\Users\\ASUS/nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\ASUS\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ASUS\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [18]:
y_train_bow = reviews_train['type']
y_test_bow = reviews_test['type']

In [20]:
y_test_bow.value_counts() / y_test_bow.shape[0]

type
Negative      0.299190
Positive      0.282252
Neutral       0.245632
Irrelevant    0.172926
Name: count, dtype: float64

In [ ]:
model1 = LogisticRegression(C=1, solver="liblinear", max_iter=200)
model1.fit(X_train_bow, y_train_bow)

test_pred = model1.predict(X_test_bow)
accuracy_score(y_test_bow, test_pred) * 100

In [ ]:
X_val_bow = bow_count.transform(val_data['lower'])
y_val_bow = val_data['type']

In [ ]:
val_pred = model1.predict(X_val_bow)
accuracy_score(y_val_bow, val_pred) * 100

In [ ]:
bow_count = CountVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1, 4)
)

X_train_bow = bow_count.fit_transform(reviews_train['lower'])
X_test_bow = bow_count.transform(reviews_test['lower'])
X_val_bow = bow_count.transform(val_data['lower'])

In [ ]:
model2 = LogisticRegression(C=0-9, solver="liblinear", max_iter=1500)
model2.fit(X_train_bow, y_train_bow)
test_pred_2 = model2.predict(X_test_bow)
accuracy_score(y_test_bow, test_pred_2) * 100

In [ ]:
val_pred_2 = model2.predict(X_val_bow)
accuracy_score(y_val_bow, val_pred_2) * 100

In [ ]:
le = LabelEncoder()
y_train_bow_num = le.fit_transform(y_train_bow)
y_test_bow_num = le.transform(y_test_bow)
y_val_bow_num = le.transform(y_val_bow)

In [ ]:
XGB = XGBClassifier(objective="multi:softmax",n_estimators=1000,colsample_bytree=0.6, subsample=0.6)
XGB.fit(X_train_bow, y_train_bow_num)

test_pred_2 = XGB.predict(X_test_bow)
accuracy_score(y_test_bow_num, test_pred_2) * 100

In [ ]:
val_pred_2 = XGB.predict(X_val_bow)
accuracy_score(y_val_bow_num, val_pred_2) * 100